In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [36]:
ACCOUNT_KEY = "Ukbgwwg+pzkJgnz15rDQiRfWRUQdbmCz89/eUP/pIDltnbsIz0/hPRMQBQSSew+zON6VQ97ZdHrt+AStxHRYgw=="
CONNECTION_STR = "DefaultEndpointsProtocol=https;AccountName=sparkpocdatastorage;AccountKey=Ukbgwwg+pzkJgnz15rDQiRfWRUQdbmCz89/eUP/pIDltnbsIz0/hPRMQBQSSew+zON6VQ97ZdHrt+AStxHRYgw==;EndpointSuffix=core.windows.net"
STORAGE_ACCT_NAME = "sparkpocdatastorage"
CONTAINER_NAME = "csvdatastore"

In [3]:
options = {'account_key':ACCOUNT_KEY}

In [4]:
df = pd.read_csv('abfs://'+CONTAINER_NAME+'@'+STORAGE_ACCT_NAME+'.dfs.core.windows.net/heart_data.csv', storage_options=options)

In [5]:
df.head(5)

,index,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [6]:
spark = SparkSession.builder.appName("sparkdev-adlfs-integration").master("local[*]").getOrCreate()

23/05/01 09:40:35 WARN Utils: Your hostname, Pavans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.100 instead (on interface en0)
23/05/01 09:40:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/01 09:40:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark_df = spark.createDataFrame(df)

/Users/pavanmantha/miniconda3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Users/pavanmantha/miniconda3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [8]:
spark_df.show(5, truncate=False)

+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|index|id |age  |gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|0    |0  |18393|2     |168   |62.0  |110  |80   |1          |1   |0    |0   |1     |0     |
|1    |1  |20228|1     |156   |85.0  |140  |90   |3          |1   |0    |0   |1     |1     |
|2    |2  |18857|1     |165   |64.0  |130  |70   |3          |1   |0    |0   |0     |1     |
|3    |3  |17623|2     |169   |82.0  |150  |100  |1          |1   |0    |0   |1     |1     |
|4    |4  |17474|1     |156   |56.0  |100  |60   |1          |1   |0    |0   |0     |0     |
+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
only showing top 5 rows



In [9]:
# Find Count of Null, None, NaN of All DataFrame Columns
spark_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|index| id|age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+-----+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|    0|  0|  0|     0|     0|     0|    0|    0|          0|   0|    0|   0|     0|     0|
+-----+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+



In [10]:
columns = spark_df.columns

columns.remove('id')
columns.remove('index')

input_columns = columns[:11]
target_column = columns[:-1]

In [11]:
input_columns

['age',
 'gender',
 'height',
 'weight',
 'ap_hi',
 'ap_lo',
 'cholesterol',
 'gluc',
 'smoke',
 'alco',
 'active']

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
vector_assembler = VectorAssembler(inputCols=input_columns, outputCol="features", handleInvalid="skip")
transformed_df = vector_assembler.transform(spark_df)
transformed_df.show(5, truncate=False)

+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+--------------------------------------------------------+
|index|id |age  |gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|features                                                |
+-----+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+--------------------------------------------------------+
|0    |0  |18393|2     |168   |62.0  |110  |80   |1          |1   |0    |0   |1     |0     |[18393.0,2.0,168.0,62.0,110.0,80.0,1.0,1.0,0.0,0.0,1.0] |
|1    |1  |20228|1     |156   |85.0  |140  |90   |3          |1   |0    |0   |1     |1     |[20228.0,1.0,156.0,85.0,140.0,90.0,3.0,1.0,0.0,0.0,1.0] |
|2    |2  |18857|1     |165   |64.0  |130  |70   |3          |1   |0    |0   |0     |1     |[18857.0,1.0,165.0,64.0,130.0,70.0,3.0,1.0,0.0,0.0,0.0] |
|3    |3  |17623|2     |169   |82.0  |150  |100  |1          |1   |0    |0   |1     |1     |[17623.0

In [14]:
required_df = transformed_df.drop('index', 'id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active')
required_df.show(5, truncate=False)

+------+--------------------------------------------------------+
|cardio|features                                                |
+------+--------------------------------------------------------+
|0     |[18393.0,2.0,168.0,62.0,110.0,80.0,1.0,1.0,0.0,0.0,1.0] |
|1     |[20228.0,1.0,156.0,85.0,140.0,90.0,3.0,1.0,0.0,0.0,1.0] |
|1     |[18857.0,1.0,165.0,64.0,130.0,70.0,3.0,1.0,0.0,0.0,0.0] |
|1     |[17623.0,2.0,169.0,82.0,150.0,100.0,1.0,1.0,0.0,0.0,1.0]|
|0     |[17474.0,1.0,156.0,56.0,100.0,60.0,1.0,1.0,0.0,0.0,0.0] |
+------+--------------------------------------------------------+
only showing top 5 rows



In [15]:
required_df.select(col("cardio")).distinct().show(truncate=False)

+------+
|cardio|
+------+
|0     |
|1     |
+------+



In [16]:
(train_set, test_set) = required_df.randomSplit([0.8,0.2])

In [17]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
dt = DecisionTreeClassifier(featuresCol='features', labelCol='cardio')
model = dt.fit(train_set)

In [19]:
predictions = model.transform(test_set)
predictions.show(10, truncate=False)

+------+-------------------------------------------------------+----------------+----------------------------------------+----------+
|cardio|features                                               |rawPrediction   |probability                             |prediction|
+------+-------------------------------------------------------+----------------+----------------------------------------+----------+
|0     |[14321.0,2.0,172.0,83.0,130.0,80.0,1.0,1.0,1.0,0.0,1.0]|[1499.0,1376.0] |[0.5213913043478261,0.4786086956521739] |0.0       |
|0     |[14333.0,2.0,172.0,81.0,120.0,80.0,1.0,1.0,0.0,0.0,1.0]|[14927.0,4255.0]|[0.7781774580335732,0.22182254196642687]|0.0       |
|0     |[14359.0,2.0,169.0,68.0,110.0,80.0,1.0,1.0,0.0,0.0,1.0]|[14927.0,4255.0]|[0.7781774580335732,0.22182254196642687]|0.0       |
|0     |[14373.0,1.0,169.0,67.0,120.0,80.0,1.0,1.0,0.0,0.0,1.0]|[14927.0,4255.0]|[0.7781774580335732,0.22182254196642687]|0.0       |
|0     |[14400.0,1.0,153.0,70.0,120.0,70.0,1.0,1.0,0.0,0.0,1.0

In [20]:
evaluator = MulticlassClassificationEvaluator(labelCol='cardio', predictionCol='prediction')
accuracy = evaluator.evaluate(predictions)
print('Accuracy=', accuracy)
print('Test Error=', 1.0-accuracy)

Accuracy= 0.7263979934409468
Test Error= 0.2736020065590532


In [21]:
rfc = RandomForestClassifier(featuresCol='features',labelCol='cardio', numTrees=5)
model_rfc = rfc.fit(train_set)

In [22]:
predictions_rfc = model_rfc.transform(test_set)
predictions_rfc.show(5, truncate=False)

+------+-------------------------------------------------------+---------------------------------------+----------------------------------------+----------+
|cardio|features                                               |rawPrediction                          |probability                             |prediction|
+------+-------------------------------------------------------+---------------------------------------+----------------------------------------+----------+
|0     |[14321.0,2.0,172.0,83.0,130.0,80.0,1.0,1.0,1.0,0.0,1.0]|[3.1677591179222824,1.8322408820777176]|[0.6335518235844565,0.3664481764155435] |0.0       |
|0     |[14333.0,2.0,172.0,81.0,120.0,80.0,1.0,1.0,0.0,0.0,1.0]|[3.7710849908724042,1.2289150091275962]|[0.7542169981744808,0.24578300182551924]|0.0       |
|0     |[14359.0,2.0,169.0,68.0,110.0,80.0,1.0,1.0,0.0,0.0,1.0]|[3.881246162177815,1.1187538378221855] |[0.7762492324355629,0.22375076756443707]|0.0       |
|0     |[14373.0,1.0,169.0,67.0,120.0,80.0,1.0,1.0,0.0,0.0

In [23]:
evaluator = MulticlassClassificationEvaluator(labelCol='cardio', predictionCol='prediction')
accuracy = evaluator.evaluate(predictions_rfc)
print('Accuracy=', accuracy)
print('Test Error=', 1.0-accuracy)

Accuracy= 0.7286927889575063
Test Error= 0.27130721104249367


In [37]:
import shutil

# save the sparkml model
model.save("decision_tree_classifier")
model_rfc.save("random_forest_classifier")

#zip the model folder
shutil.make_archive("decision_tree_classifier", "zip", "decision_tree_classifier")
shutil.make_archive("random_forest_classifier", "zip", "random_forest_classifier")

#delete the actual folder
shutil.rmtree("decision_tree_classifier")
shutil.rmtree("random_forest_classifier")

## Upload Model back to Azure Blob Storage

In [39]:
from azure.storage.blob import BlobServiceClient

CONTAINER_NAME = "trained-models"

def upload_file(file_path, file_name):
    blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STR)
    blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=file_name)

    with open(file_path, "rb") as file:
        blob_client.upload_blob(file)

    print(f"File upload complete to container {CONTAINER_NAME}")


upload_file("/Users/pavanmantha/Pavans/PracticeExamples/DataScience_Practice/spark-handson/AzureBlobStorage_Spark_Integration/decision_tree_classifier.zip", "decision_tree_classifier.zip")

upload_file("/Users/pavanmantha/Pavans/PracticeExamples/DataScience_Practice/spark-handson/AzureBlobStorage_Spark_Integration/random_forest_classifier.zip", "random_forest_classifier.zip")


File upload complete to container trained-models
File upload complete to container trained-models
